In [1]:
import torch
import numpy as np

# 定义参数
batch_size = 400
input_size = 1

# 定义随机性参数
sequence_lengths = [50, 100, 200, 500, 800]  # 可选的采样点数
period_range = (1, 100)  # 随机选择周期范围

# 初始化用于存储所有信号的列表
all_clean_batches = []
all_noisy_batches = []

# 生成多种随机信号并融合
for sequence_length in sequence_lengths:
    period = np.random.uniform(*period_range)  # 随机选择周期

    # 生成干净信号
    clean_signal = torch.sin(
        torch.linspace(0, 2 * np.pi * period, batch_size * sequence_length)
    ).unsqueeze(-1)   # 添加最后一维，使形状为 (batch_size * sequence_length, 1)

    # 检查生成信号的长度是否能整齐划分为 batch_size
    total_points = batch_size * sequence_length
    if clean_signal.size(0) != total_points:
        print(f"Skipping sequence length {sequence_length} due to incompatible size.")
        continue

    # 划分为批次形式
    clean_batches = clean_signal.view(batch_size, sequence_length, input_size)

    # 生成噪声信号
    noise = torch.randn_like(clean_signal) * 0.2  # 调整噪声强度
    noisy_signal = clean_signal + noise
    noisy_batches = noisy_signal.view(batch_size, sequence_length, input_size)

    # 将生成的数据添加到列表中
    all_clean_batches.append((sequence_length, clean_batches))
    all_noisy_batches.append((sequence_length, noisy_batches))

# 确保序列长度相同的批次分别拼接
final_clean_batches = []
final_noisy_batches = []

for sequence_length in sequence_lengths:
    clean_batches_group = [cb for sl, cb in all_clean_batches if sl == sequence_length]
    noisy_batches_group = [nb for sl, nb in all_noisy_batches if sl == sequence_length]

    if clean_batches_group and noisy_batches_group:
        final_clean_batches.append(torch.cat(clean_batches_group, dim=0))
        final_noisy_batches.append(torch.cat(noisy_batches_group, dim=0))

# 打印拼接结果
for i, sequence_length in enumerate(sequence_lengths):
    if i < len(final_clean_batches):
        print(f"Final clean batches for sequence length {sequence_length}: {final_clean_batches[i].size()}")
        print(f"Final noisy batches for sequence length {sequence_length}: {final_noisy_batches[i].size()}")


Final clean batches for sequence length 50: torch.Size([400, 50, 1])
Final noisy batches for sequence length 50: torch.Size([400, 50, 1])
Final clean batches for sequence length 100: torch.Size([400, 100, 1])
Final noisy batches for sequence length 100: torch.Size([400, 100, 1])
Final clean batches for sequence length 200: torch.Size([400, 200, 1])
Final noisy batches for sequence length 200: torch.Size([400, 200, 1])
Final clean batches for sequence length 500: torch.Size([400, 500, 1])
Final noisy batches for sequence length 500: torch.Size([400, 500, 1])
Final clean batches for sequence length 800: torch.Size([400, 800, 1])
Final noisy batches for sequence length 800: torch.Size([400, 800, 1])
